Reading classics [Deep Learning Models](https://nbviewer.jupyter.org/github/rasbt/deeplearning-models/blob/master/pytorch_ipynb/autoencoder/ae-basic.ipynb)

## Code Modules, Functions, & Classes

[Google Colaboratory Variant](https://colab.research.google.com/drive/1kydV6_IC0BtHwpijx4oVjEab_Z39RpCQ)

In [ ]:
import numpy as np,pandas as pd,pylab as pl
import h5py,torch
from torchvision.datasets import MNIST as tmnist
from torchvision import transforms,utils
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn.functional as tnnf
import torch.nn as tnn
import tensorflow.image as timage
dev=torch.device("cuda:0" \
if torch.cuda.is_available() else "cpu")

In [ ]:
class TData(tds):
    def __init__(self,x,y):   
        self.x=torch.tensor(x,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.int32)
    def __getitem__(self,index):
        img,lbl=self.x[index],self.y[index]
        return img,lbl
    def __len__(self):
        return self.y.shape[0]

In [ ]:
def display_examples(data_loader,img_size):
    for images,labels in data_loader:  
        print('Image dimensions: %s'%str(images.shape))
        print('Label dimensions: %s'%str(labels.shape))
        n=np.random.randint(1,50)
        fig=pl.figure(figsize=(10,4))
        for i in range(n,n+5):
            ax=fig.add_subplot(1,5,i-n+1,\
            xticks=[],yticks=[],title=labels[i].item())
            ax.imshow((images[i]).reshape(img_size,img_size),
                      cmap=pl.cm.bone)
        break
def display_results(num_images):
    num_images=int(num_images)
    fig,axes=pl.subplots(nrows=2,ncols=num_images, 
                         sharex=True,sharey=True,
                         figsize=(10,4))
    original_images=features[:num_images]
    decoded_images=decoded[:num_images]
    for i in range(num_images):
        for ax,img in zip(axes,[original_images,
                                decoded_images]):
            curr_img=img[i].detach().to(torch.device('cpu'))
            ax[i].imshow(curr_img.view((img_size,img_size)),
                         cmap='bone')
    pl.show()

## Data

In [ ]:
random_seed=12; batch_size=128; img_size=32
trans=transforms\
.Compose([transforms.Resize((img_size,img_size)),
          transforms.ToTensor()])
train=tmnist(root='data',train=True,
             download=True,transform=trans)
test=tmnist(root='data',train=False, 
            transform=trans)
train_loader=tdl(dataset=train,shuffle=True, 
                 batch_size=batch_size)
test_loader=tdl(dataset=test,shuffle=False, 
                batch_size=batch_size)

In [ ]:
display_examples(test_loader,img_size)

In [ ]:
fpath='../input/classification-of-handwritten-letters/'
f='LetterColorImages_123.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x=1-np.array(f[keys[1]],dtype='float32')/255
x=timage.resize(x,[img_size,img_size])
x=(np.dot(x.numpy(),[.299,.587,.114]))\
.reshape(-1,1,img_size,img_size)
y=np.array(f[keys[2]],dtype='int32')-1
N=y.shape[0]; n=int(.2*N)
shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
x,y=x[shuffle_ids],y[shuffle_ids]
x_test,x_train=x[:n],x[n:]
y_test,y_train=y[:n],y[n:]

In [ ]:
random_seed=23; batch_size2=128
train2=TData(x_train,y_train)
test2=TData(x_test,y_test)
train_loader2=tdl(dataset=train2,shuffle=True,
                  batch_size=batch_size2)
test_loader2=tdl(dataset=test2,shuffle=True,
                 batch_size=batch_size2)

In [ ]:
display_examples(test_loader2,img_size)

## Autoencoder
compresses 32x32-pixel images into (hidden1)-pixel vectors & restores the original images

In [ ]:
class AE(tnn.Module):
    def __init__(self,num_features,hidden1):
        super(AE,self).__init__()
        # encoder
        self.linear1=tnn.Linear(num_features,hidden1)
        self.linear1.weight.detach().normal_(0.,.1)
        self.linear1.bias.detach().zero_()
        # decoder
        self.linear2=tnn.Linear(hidden1,num_features)
        self.linear2.weight.detach().normal_(0.,.1)
        self.linear2.bias.detach().zero_()
    def forward(self,x):        
        # encoder
        encoded=self.linear1(x)
        encoded=tnnf.leaky_relu(encoded)     
        # decoder
        logits=self.linear2(encoded)
        decoded=torch.sigmoid(logits)
        return decoded

## Training

In [ ]:
torch.manual_seed(random_seed)
learning_rate=.0005; hidden1=32
model=AE(num_features=img_size**2,hidden1=hidden1)
model=model.to(dev)
optimizer=torch.optim.Adam(model.parameters(),
                           lr=learning_rate)

In [ ]:
epochs=10
for epoch in range(epochs):
    for batch_ids,(features,targets) in enumerate(train_loader):
        features=features.view(-1,img_size**2).to(dev)
        decoded=model(features)
        cost=tnnf.binary_cross_entropy(decoded,features)
        optimizer.zero_grad()      
        cost.backward(); optimizer.step()
        if not batch_ids%200:
            print ('Epoch: %03d/%03d | Batch: %03d/%03d | Cost: %.4f' 
                   %(epoch+1,epochs,batch_ids,
                     len(train_loader),cost))

In [ ]:
display_results(5)

## Autoencoder 2
compresses 32x32-pixel images with convolutional layers & restores the original images

In [ ]:
class AE2(tnn.Module):
    def __init__(self):        
        super(AE2,self).__init__()
        # encoder
        self.conv1=tnn.Conv2d(in_channels=1,out_channels=4,
                              kernel_size=(3,3),stride=(1,1),
                              padding=1)
        self.pool1=tnn.MaxPool2d(kernel_size=(2,2),
                                 stride=(2,2),padding=0)                                       
        self.conv2=tnn.Conv2d(in_channels=4,out_channels=16,
                              kernel_size=(3,3),stride=(1,1),
                              padding=1)                                         
        self.pool2=tnn.MaxPool2d(kernel_size=(2,2),
                                 stride=(2,2),padding=0)
        # decoder
        self.conv3=tnn.Conv2d(in_channels=16,out_channels=4,
                              kernel_size=(3,3),stride=(1,1),
                              padding=1)
        self.conv4=tnn.Conv2d(in_channels=4,out_channels=1,
                              kernel_size=(3,3),stride=(1,1),
                              padding=1)       
    def forward(self,x):        
        # encoder
        x=self.conv1(x); x=tnnf.leaky_relu(x)
        x=self.pool1(x)
        x=self.conv2(x); x=tnnf.leaky_relu(x)
        x=self.pool2(x)        
        # decoder
        x=tnnf.interpolate(x,scale_factor=2,mode='nearest')
        x=self.conv3(x)
        x=tnnf.leaky_relu(x)
        x=tnnf.interpolate(x,scale_factor=2,mode='nearest')
        x=self.conv4(x)
        logits=tnnf.leaky_relu(x)
        probs=torch.sigmoid(logits)
        return logits,probs

## Training

In [ ]:
torch.manual_seed(random_seed)
learning_rate=.05
model=AE2()
model=model.to(dev)
optimizer2=torch.optim.Adam(model.parameters(),
                            lr=learning_rate) 

In [ ]:
epochs=100
for epoch in range(epochs):
    for batch_ids,(features,targets) in enumerate(train_loader2):
        features=features.to(dev)
        logits,decoded=model(features)
        cost=tnnf.binary_cross_entropy_with_logits(logits,features)
        optimizer2.zero_grad()      
        cost.backward(); optimizer2.step()
        if not batch_ids%100:
            print ('Epoch: %03d/%03d | Batch: %03d/%03d | Cost: %.4f' 
                   %(epoch+1,epochs,batch_ids,
                     len(train_loader2),cost))

In [ ]:
display_results(5)